In [ ]:
%pip install -q -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 1
from imports import *
from functions import *
from database import engine, metadata

# Get Available Stations ID List

In [ ]:
# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()
# parse the response as a dictionary
stations_df = api.stations(as_df=True)

print(len(stations_df))

# Filter Buoys by Remarks

In [ ]:
access_error_url_list = []

# Liste de mots à rechercher dans la colonne "Remark"
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift']
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0]}')

# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

# Parcours des lignes de la DataFrame
for idx, row in stations_df.iterrows():
    station_id = row["Station"]
    station_Location = row["Hull No./Config and Location"]  # Extraire la valeur de la cellule pour chaque ligne
    
    # Extraction du nom de la station si un ")" est trouvé
    if ")" in station_Location:
        station_name = station_Location.split(')')[1].rstrip(" )")  # On enlève l'espace et la parenthèse en fin de chaîne
    else:
        station_name = station_Location.strip()  # Si pas de ")", on garde toute la chaîne

    station_name = station_name.rstrip(" )").replace("(", "").replace(")", "").strip()

    # Nettoyage final pour enlever toute parenthèse ou espace en fin de nom
    station_name = station_name.rstrip(" )")

    # Vérifier si "Remark" n'est pas NaN et si un des éléments de blacklist est dans "Remark"
    if isinstance(row["Remark"], str) and any(blacklist_word.lower() in row["Remark"].lower() for blacklist_word in blacklist):
        # Ajouter l'index à la liste
        indices_a_supprimer.append(idx)
        url = get_buoy_url(station_id)
        access_error_url_list.append(url)
    else:
        pass
# Supprimer les lignes après la boucle
stations_df.drop(index=indices_a_supprimer, inplace=True)

print(f'Après Filtre: {stations_df.shape[0]}')

# Build Buoys_datas Dict

In [ ]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}
buoy_list = []

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']
    metadata = get_station_metadata(buoy_id)

    # ✅ Récupérer les données sous forme de dictionnaire
    buoy_info = parse_buoy_json(metadata)

    # ✅ Stocker directement les données dans buoy_datas
    buoy_datas[buoy_id] = buoy_info
    buoy_list.append(buoy_id)

# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}\n")

# Afficher le contenu de buoy_datas

first_key =next(iter(buoy_datas))
first_key
buoy_datas[first_key]

# Collecte de données marines et météos

In [ ]:
# 🚀 Démarrage du processus
print("\n🚀 Démarrage du processus de collecte des données...\n")

# Initialisation des compteurs
marine_data_collected_successfully = marine_data_collected_failed = 0
meteo_data_collected_successfully = meteo_data_collected_failed = 0

success = False
total_stations = stations_df.shape[0]
count = 0

# 🔄 Parcours des bouées / stations
for idx, row in stations_df.iterrows():
    buoy_id = row["Station"]

    ######### 🌊 MARINE DATA #########
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            marine_data_collected_failed += 1
            continue

        marine_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Ajout des métadonnées
    try:
        buoy_info = buoy_datas.get(buoy_id, {})
        Lat, Lon = buoy_info.get('lat_buoy'), buoy_info.get('lon_buoy')
        if Lat is None or Lon is None:
            raise ValueError(f"Données manquantes pour {buoy_id}")

        df_marine['Lat'] = Lat
        df_marine['Lon'] = Lon
        df_marine['Water_depth'] = buoy_info.get('Water_depth', None)
        df_marine.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_marine.columns]
        df_marine['Datetime'] = df_marine['Datetime'].dt.tz_localize(None)

        buoy_datas[buoy_id]["Marine"] = df_marine

        station_zone = safe_get(parse_buoy_json(get_station_metadata(buoy_id)), "station_zone")
        Bronze_Marine_table_Name = f"br_{buoy_id}_marine_{station_zone}".replace('.', '_').replace('-', '_').replace(' ', '_').lower()

    except Exception as e:
        print(f"⚠️ Erreur métadonnées marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    ######### ⛅ METEO DATA #########
    try:
        df_meteo = meteo_api_request([Lat, Lon])
        if df_meteo is None or df_meteo.empty:
            meteo_data_collected_failed += 1
            continue
        
        rename_columns(df_meteo, {'date':'Datetime'})
        df_meteo.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_meteo.columns]
        df_meteo['Datetime'] = df_meteo['Datetime'].dt.tz_localize(None)
    
        buoy_datas[buoy_id]["Meteo"] = df_meteo
        meteo_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte météo {buoy_id}: {e}")
        meteo_data_collected_failed += 1
        continue

# Retirer les bouées avec des DataFrames vides ou None
buoy_datas = {buoy_id: data for buoy_id, data in buoy_datas.items() 
              if "Marine" in data and data["Marine"] is not None and not data["Marine"].empty
              and "Meteo" in data and data["Meteo"] is not None and not data["Meteo"].empty}

# 🔚 Résumé final

print("\n📝 Résumé final :")
print(f"🌊 Marine - Collecte ✅ {marine_data_collected_successfully} ❌ {marine_data_collected_failed}")
print(f"⛅ Météo - Collecte ✅ {meteo_data_collected_successfully} ❌ {meteo_data_collected_failed}")

# Afficher la longueur du dictionnaire (nombre de bouées avec des données valides)
print(f"\n📊 Nombre de bouées avec des données valides : {len(buoy_datas)}")

# Data Enrichment with MetaDatas

In [ ]:
list_not_include = ['lon_buoy', "lat_buoy", "url"]
for buoy_id, value in buoy_datas.items():
    print(f"\n🔍 Traitement de la Station ID: {buoy_id}")

    marine_df = buoy_datas[buoy_id]["Marine"]
    meteo_df = buoy_datas[buoy_id]["Meteo"]

    try:
        # Récupérer les métadonnées de la station
        buoy_metadata = get_station_metadata(buoy_id)
        parsed_data = parse_buoy_json(buoy_metadata)

        # Mise à jour du dictionnaire avec les métadonnées
        data = buoy_datas[buoy_id]
        data.update(parsed_data)
        
        # Ajouter les métadonnées comme nouvelles colonnes dans marine_df
        if marine_df is not None:
            marine_df["Station ID"] = str(buoy_id)
            for key, value in parsed_data.items():
                # Vérifier si la clé n'est pas dans la liste des exclusions
                if key not in list_not_include:
                    marine_df[key] = value
                    print(f"✅ Colonne '{key}' ajoutée au DataFrame de la station {buoy_id}")

    except Exception as e:
        print(f"❌ Erreur pour la station {buoy_id}: {e}")

# Vérification de l'ajout des colonnes en prenant un id au hasard
station_id = random.choice(list(buoy_datas.keys()))
marine_df = buoy_datas[station_id]["Marine"]

if marine_df is not None:
    print("\nColonnes ajoutées au DataFrame de la station", station_id)
    print(marine_df.columns)

In [ ]:
display_buoys_missing_df_counts(buoy_datas)

In [ ]:
display(df_marine.columns)
display(df_meteo.columns)

# Handle Null Values

In [ ]:
important_columns_oceanography = [
    'wind_direction',             
    'wind_speed',                 
    'wave_height',                   
    'pressure',                   
    'air_temperature',            
    'water_temperature',          
    'Datetime',
    'Lat',
    'Lon'                 
]

important_columns_meteorology = [
    'temperature_2m',             
    'relative_humidity_2m',       
    'dew_point_2m',               
    'precipitation',              
    'pressure_msl',               
    'cloud_cover',                
    'wind_speed_10m',             
    'Datetime'
]

stations_depart = len(buoy_datas)
ignored_buoys = {}  # Dictionary to track ignored buoys and their reasons

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Nettoyage des données pour la station {station_id}")

    marine_df = data.get("Marine")
    meteo_df = data.get("Meteo")

    if marine_df is None or meteo_df is None:
        ignored_buoys[station_id] = "Marine DataFrame ou Meteo DataFrame manquant(e)"
        print(f"⚠️ Station {station_id} ignorée: Marine DataFrame ou Meteo DataFrame manquant(e)")
        continue

    try:
        # Nettoyage des DataFrames
        cleaned_marine_df = handle_null_values(marine_df)
        cleaned_meteo_df = handle_null_values(meteo_df)
        # Vérification des colonnes importantes après nettoyage
        marine_columns_ok = all(col in cleaned_marine_df.columns for col in important_columns_oceanography)
        meteo_columns_ok = all(col in cleaned_meteo_df.columns for col in important_columns_meteorology)

        # Track which columns are missing
        missing_marine_columns = [col for col in important_columns_oceanography if col not in cleaned_marine_df.columns]
        missing_meteo_columns = [col for col in important_columns_meteorology if col not in cleaned_meteo_df.columns]

        if missing_marine_columns or missing_meteo_columns:
            ignored_buoys[station_id] = f"Colonnes manquantes: Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}"
            print(f"⚠️ Station {station_id} ignorée: Colonnes manquantes - Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}")
            continue

        # Ajouter le DataFrame nettoyé au dictionnaire des résultats
        buoy_datas[station_id]['Cleaned Marine'] = cleaned_marine_df
        buoy_datas[station_id]['Cleaned Meteo'] = cleaned_meteo_df
        print(f"✅ Nettoyage réussi pour la station {station_id} ({cleaned_marine_df.shape[0]} lignes)")

    except Exception as e:
        ignored_buoys[station_id] = f"Erreur lors du nettoyage: {e}"
        print(f"❌ Erreur lors du nettoyage pour {station_id}: {e}")

# 🔥 Suppression des stations ignorées du dictionnaire principal
for station_id in ignored_buoys:
    buoy_datas.pop(station_id, None)

len_cleaned_data = len([data for data in buoy_datas.values() if 'Cleaned Marine' in data and 'Cleaned Meteo' in data])

# Résumé final du nettoyage
print("\n📊 RÉSUMÉ DU NETTOYAGE:")
print(f"📌 Stations au départ : {stations_depart}")
print(f"✅ Stations nettoyées : {len_cleaned_data}")
print(f"🏁 Stations restantes après filtrage :")

for station_id, reason in ignored_buoys.items():
    print(f"🛑 Station {station_id} ignorée: {reason}")

print(f"\n🧹 Clés restantes dans buoy_datas après purge : {len(buoy_datas)} (attendu : {len_cleaned_data})")

In [ ]:
display_buoys_missing_df_counts(buoy_datas, prefix="Cleaned")

In [ ]:
display_row_values(df_meteo)

In [ ]:
# Définir la taille de la figure
plt.figure(figsize=(12, 6))

# Utiliser seaborn pour créer la heatmap des valeurs manquantes
sns.heatmap(df_meteo.isnull(), 
            cbar=False, 
            cmap='viridis', 
            yticklabels=False)

plt.title('Carte thermique des valeurs manquantes dans df_meteo')
plt.xlabel('Colonnes')
plt.ylabel('Entrées')

# Fusionner les df_meteo et df_marine sur 'Datetime'

In [ ]:
# Fusion des DataFrames nettoyés
print("\n🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION")

merged_success_count = 0  # Compteur de fusions réussies
total_merged_rows = 0     # Total de lignes fusionnées

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Fusion des données pour la station {station_id}")

    cleaned_marine_df = data.get("Cleaned Marine")
    cleaned_meteo_df = data.get("Cleaned Meteo")

    if cleaned_marine_df is None or cleaned_meteo_df is None:
        continue

    try:
        merged_df = pd.merge(cleaned_marine_df, cleaned_meteo_df, on="Datetime", how="inner")

        if merged_df.empty:
            print(f"⚠️ Station {station_id} fusionnée, mais résultat vide après inner merge sur 'Datetime'")
        else:
            buoy_datas[station_id]["Merged"] = merged_df
            merged_success_count += 1
            total_merged_rows += len(merged_df)
            print(f"✅ Fusion réussie pour la station {station_id} ({merged_df.shape[0]} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la fusion pour {station_id}: {e}")

# Résumé des fusions
print(f"\n📦 Fusions réussies : {merged_success_count}/{len_cleaned_data} stations")
print(f"📊 Total de lignes fusionnées : {total_merged_rows}")

# Concaténation des DataFrames fusionnés

In [ ]:
# Concaténation des DataFrames fusionnés
print("\n🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME")

final_merged_df_list = []
concat_success_count = 0
concat_total_rows = 0

for station_id, data in buoy_datas.items():
    merged_df = data.get("Merged")

    if merged_df is None:
        print(f"⚠️ Station {station_id} ignorée pour concaténation: Données fusionnées manquantes")
        continue

    try:
        final_merged_df_list.append(merged_df)
        concat_success_count += 1
        concat_total_rows += len(merged_df)
        print(f"✅ Concaténation réussie pour la station {station_id} ({len(merged_df)} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la concaténation pour {station_id}: {e}")

# Création du DataFrame final unique
try:
    df_final = pd.concat(final_merged_df_list, ignore_index=True)
    print(f"\n🧾 DataFrame final créé avec succès ({df_final.shape[0]} lignes, {df_final.shape[1]} colonnes)")
except Exception as e:
    print(f"\n❌ Erreur lors de la création du DataFrame final: {e}")
    df_final = None

# Résumé
print(f"\n📦 Concaténations réussies : {concat_success_count}/{merged_success_count}")
print(f"📊 Total de lignes dans le DataFrame final : {concat_total_rows}")

In [ ]:
display_row_values(df_final)

In [ ]:
df_final.index

In [ ]:
df_final.dtypes

Null values heatmap

In [ ]:
# null value heatmap with sns

plt.figure(figsize=(16, 10))

sns.heatmap(df_final.isnull(), cbar=False, cmap='viridis')

plt.title('Null Values Heatmap', fontdict={'size': 20})

plt.show()

In [ ]:
df_final = handle_null_values(df_final)
display_row_values(df_final)

# Hour Filtering

In [ ]:
try:
    df_final = df_final[['Datetime', 'Lat', 'Lon'] + [col for col in df_final.columns if col not in ['Datetime', 'Lat', 'Lon']]]
    # placer la colonne Datetime en %Y-%m-%d %H
    
    print(f"🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: {df_final.shape[0]} lignes")

except Exception as e:
    print(str(e))
finally:
    display(df_final.columns)

In [ ]:
df_final.columns = [col.strip() for col in df_final.columns]

# Renaming and deleting useless columns

In [ ]:
# Dictionnaire de renommage des colonnes
col_to_rename = {'temperature_2m': 'T°(C°)', 
                 'relative_humidity_2m': 'Relative Humidity (%)',
                 'dew_point_2m': 'Dew Point (°C)', 
                 'precipitation': 'Precipitations (mm)',  
                 'pressure_msl':'Sea Level Pressure (hPa)', 
                 'cloud_cover_low':'Low Clouds (%)',
                 'cloud_cover_mid' : 'Middle Clouds (%)', 
                 'cloud_cover_high' : 'High Clouds (%)', 
                 'visibility' : 'Visibility (km)', 
                 'wind_direction': 'Wind Direction (°)',
                 'wind_speed': 'Wind Speed (km/h)', 
                 'wind_gust': 'Wind Gusts (km/h)',
                 'wind_speed_10m':'Wind Speed (10m)', 
                 'surface_pressure': 'Surface Pressure',
                 'wave_height': 'Wave Height (m)', 
                 'average_wave_period': 'Average Wave Period (s)',
                 'dominant_wave_direction': 'Dominant Wave Direction (°)', 
                 'pressure': 'Pressure (hPa)',
                 'air_temperature': 'Air T°', 
                 'water_temperature': 'Water T°', 
                 'Water_depth': 'Water Depth (m)', 
                 "Air_temp_height": "Air T° Height", 
                 "Anemometer_height": "Anemometer Height (m)", 
                 "station_zone": "Station Zone",
                 "Barometer_elevation": "Barometer Elevation", 
                 "sea_temp_depth" : "Sea Temperature Depth (m)",
                 "cloud_cover": "Cloud Cover (%)"
                 }

# Liste des colonnes à supprimer
cols_to_delete = ['soil_temperature_0cm', 'lat_buoy','lon_buoy', 'rain', 
                  'showers', 'is_day', 'soil_moisture_0_to_1cm']
	
# Renommer les colonnes d'abord
df_final = rename_columns(df_final, col_to_rename)
# Ensuite, supprimer les colonnes non désirées
df_final = drop_columns_if_exist(df_final, cols_to_delete)
try:
    if df_final['Visibility (km)'].mean() > 1000:
        df_final['Visibility (km)'] = df_final['Visibility (km)'] / 1000
        print("Conversion de la visibilité de mètres à kilomètres")
    df_final["T°(C°)"] = round(df_final["T°(C°)"], 2)
    df_final["Wind Speed (10m)"] = round(df_final["Wind Speed (10m)"], 2)
except Exception as e:
    print(f"�� Erreur lors du traitement des colonnes :\n {e}")

# Afficher les résultats
print("\nColonnes après renommage et suppression :")
print("\n")
display_row_values(df_final)

Transformer les coordonnées en format float

In [ ]:
try:

    df_final[['Lat', 'Lon']] = df_final.apply(
        lambda row: pd.Series(convert_coordinates(row['Lat'], row['Lon'])),
        axis=1
    )
except Exception as e:
    print(f"Erreur : {e}")
finally:
    display_row_values(df_final)

Pressure and air temperatures are very close
We'll make the average of them all

In [ ]:
try:
    df_final['T°(C°)'] = (df_final['Air T°'] + df_final['T°(C°)']) / 2
    df_final['T°(C°)'] = df_final['T°(C°)'].round(2)
    df_final.drop(columns=['Air T°'], inplace=True)
    
    df_final['Sea Level Pressure (hPa)'] = round((df_final['Sea Level Pressure (hPa)'] + df_final['Surface Pressure']) / 2, 2)
    df_final.drop(columns=['Surface Pressure'], inplace=True)
except Exception as e:
    print(f"Erreur :\n {e}")
    
display_row_values(df_final)

Create Categorical Time columns and delete the 'm' in Water Depth Column

In [ ]:
try:
    # Créer une colonne temporaire pour accéder à .dt

    df_final['Year'] = df_final['Datetime'].dt.year
    df_final['Month'] = df_final['Datetime'].dt.month_name()
    df_final['DayOfWeek'] = df_final['Datetime'].dt.day_name()
    df_final['DayPeriod'] = df_final['Datetime'].apply(
        lambda x: 'Morning' if 6 <= x.hour < 12 else
                  'Afternoon' if 12 <= x.hour < 18 else
                  'Evening' if 18 <= x.hour < 22 else
                  'Night'
    )

except Exception as e:
    print(f"Erreur :\n {e}")

try:
    # virer le m dans Water Depth avec regex lambda et passer la colonne en float
    df_final['Water Depth (m)'] = df_final['Water Depth (m)'].apply(lambda x: re.sub(r'\D', '', str(x)).strip())
    df_final['Water Depth (m)'] = df_final['Water Depth (m)'].astype(float)
except Exception as e:
    print(str(e))

display_row_values(df_final)

Check truth about Wind Speed Using another API call

In [ ]:
df_42058 = df_final[df_final['Station ID'] == "42058"]
df_42058.columns

# Requête à l'API Visual Crossing pour les données de vérification (1 / 24h)

In [ ]:
# ---- Chargement de la clé API ----
vc_api_key_path = r"c:\Credentials\visual_crossing_weather_api.json"
with open(vc_api_key_path, 'r') as file:
    content = json.load(file)
    vc_api_key = content["api_key"]

# ---- Extraire les coordonnées depuis la première ligne du DataFrame ----
lat_42058, lon_42058 = None, None

if not df_42058.empty:
    first_row = df_42058.iloc[0]
    lat_42058, lon_42058 = first_row["Lat"], first_row["Lon"]

# ---- Définir les dates pour la requête ----
today = datetime.now().strftime("%Y-%m-%d")
last_month = (datetime.now() - timedelta(days=31)).strftime("%Y-%m-%d")

# ---- Créer le dossier de cache si nécessaire ----
cache_dir = "api_call_files"
os.makedirs(cache_dir, exist_ok=True)

# ---- Définir le fichier cache selon la position ----
cache_file = os.path.join(cache_dir, f"vc_meteo_{lat_42058}_{lon_42058}.csv")

# ---- Vérifier si un cache récent existe (moins de 24h) ----
use_cache = False
if os.path.exists(cache_file):
    last_modified = datetime.fromtimestamp(os.path.getmtime(cache_file))
    if datetime.now() - last_modified < timedelta(hours=24):
        print(f"📦 Cache détecté ({cache_file}), modifié le {last_modified.strftime('%Y-%m-%d %H:%M:%S')}")
        vc_meteo_df = pd.read_csv(cache_file)
        print("✅ Données météo rechargées depuis le cache.")
        use_cache = True
    else:
        print(f"⚠️ Cache trouvé mais périmé (plus de 24h) → nouvelle requête API.")

# ---- Appel API si pas de cache valide ----
if not use_cache and lat_42058 is not None and lon_42058 is not None:
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat_42058},{lon_42058}/{last_month}/{today}?unitGroup=metric&key={vc_api_key}&contentType=json"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            vc_meteo_data = response.json()
            print("🌍 Données météo récupérées depuis l'API Visual Crossing.")

            # ---- Extraire les données journalières et sauvegarder ----
            if "days" in vc_meteo_data:
                vc_meteo_df = pd.json_normalize(vc_meteo_data["days"])
                vc_meteo_df.to_csv(cache_file, index=False)
                print(f"💾 Données sauvegardées dans le cache : {cache_file}")
            else:
                print("⚠️ Le champ 'days' est absent de la réponse API.")
        else:
            print(f"❌ Échec de l’appel API — code de statut : {response.status_code}")
    except Exception as e:
        print(f"❌ Exception levée lors de la requête API : {e}")
        



# 📁 Charger les données du CSV Visual Crossing
vc_csv_path = f"api_call_files/vc_meteo_{lat_42058}_{lon_42058}.csv"
df_vc_meteo = pd.read_csv(vc_csv_path)
try:
    # mettre Datetime en index
    df_vc_meteo.rename(columns={"datetime": "Datetime"}, inplace=True)  
    
except Exception as e:
    print(f"Erreur :\n {e}")

df_vc_meteo.head()

# Nettoyage du DataFrame retourné

In [ ]:
# 🧼 Nettoyage et transformation
all_hours = []

for i, row in df_vc_meteo.iterrows():
    try:
        hours_list = ast.literal_eval(row['hours'])

        for hour_data in hours_list:
            hour_data['Date'] = i  # ✅ On met l'index courant, i.e. la date du jour
            all_hours.append(hour_data)

    except Exception as e:
        print(f"Erreur parsing ligne {i}: {e}")

df_vc_flat = pd.DataFrame(all_hours)

# 🕒 Convertir le timestamp en datetime string
df_vc_flat["Datetime"] = pd.to_datetime(df_vc_flat["datetimeEpoch"], unit="s").dt.strftime("%Y-%m-%d-%H")

# 🗓️ Filtrer sur les 30 derniers jours
today = datetime.now()
thirty_days_ago = today - timedelta(days=30)

df_vc_flat['Date'] = pd.to_datetime(df_vc_flat['Date'])  # 👈 Assurer que c'est bien du datetime
df_vc_last_month = df_vc_flat[
    (df_vc_flat['Date'] >= thirty_days_ago) & 
    (df_vc_flat['Date'] <= today)
]


# Renommage des colonnes pour faciliter la comparaison

In [ ]:
#Filtrer les colonnes nécessaires
try:
    df_vc_last_month = df_vc_last_month[["Datetime", "temp", "humidity", "precip", "dew", "windgust", 
                                     "windspeed", "winddir", "pressure", "visibility"]]
except Exception as e:
        print(f"Erreur lors du filtrage des colonnes:\n {e}\n")

try:
        df_vc_last_month["Datetime"] = pd.to_datetime(df_vc_last_month["Datetime"], errors='coerce')
except Exception as e:
        print(f"Erreur lors du reformatage de la colonne Datetime:\n {e}\n")        

for col in df_vc_last_month.columns:
        try:
            # 
            if not "Datetime" in col:
                if not col.startswith("VC_"):
                        rename_columns(df_vc_last_month, {col: f"VC_{col}"})
        except Exception as e:
                print(f"Erreur lors du renommage de la colonne {col}:\n {e}\n")


Enforce Datetime Format

In [ ]:
# Check if the Datetime column is correctly converted
print(df_42058["Datetime"].dtype)
print(df_vc_last_month["Datetime"].dtype)

Merge and Compare DataFrames

In [ ]:
try:
    
    df_compare = pd.merge(df_42058, df_vc_last_month, on="Datetime", how="inner")
    display(df_compare.columns)
except Exception as e:
    print(e)

Comparaison Wind Speed

In [ ]:
try:
    # Wind Speed Comparison
    df_windspeed_compare = df_compare[['Wind Speed (km/h)', 'Wind Speed (10m)','VC_windspeed']]
    #  Pressure Comparison
    df_pressure_compare = df_compare[['Sea Level Pressure (hPa)','Pressure (hPa)', 'VC_pressure']]
    # Dew Point Comparison
    df_dew_compare = df_compare[['dewpoint','Dew Point (°C)', 'VC_dew']]
except Exception as e:
    print(e)

df_windspeed_compare.head(10)

In [ ]:
wind_col_to_delete = ['Wind Speed (km/h)', 'Anemometer Height (m)']
df_final = drop_columns_if_exist(df_final, wind_col_to_delete)
# Arrondir les valeurs de df_final

df_final = df_final.round(2)

Comparaison Pressure

In [ ]:
df_pressure_compare.head()

In [ ]:
df_final = drop_columns_if_exist(df_final, ['Pressure (hPa)'])

In [ ]:
df_dew_compare.head()

Code Custom GPT Wisdom of crowd pour le DewPoint

In [ ]:
try:

    # Calcul des distances absolues entre chaque paire de mesures pour chaque ligne
    # Ces distances nous permettent de savoir quelle mesure est la plus proche des autres
    df_compare['dist_dewpoint_DewPoint'] = np.abs(df_compare['dewpoint'] - df_compare['Dew Point (°C)'])
    df_compare['dist_dewpoint_VC'] = np.abs(df_compare['dewpoint'] - df_compare['VC_dew'])
    df_compare['dist_DewPoint_VC'] = np.abs(df_compare['Dew Point (°C)'] - df_compare['VC_dew'])

    # Pour chaque ligne, on détermine quelle mesure est la plus proche des deux autres :
    # - Si 'dewpoint' est plus proche des autres mesures que 'Dew Point (°C)' et 'VC_dew', alors 'dewpoint' est marqué comme plus proche.
    df_compare['dewpoint_closer'] = (df_compare['dist_dewpoint_VC'] < df_compare['dist_dewpoint_DewPoint']) & (df_compare['dist_dewpoint_VC'] < df_compare['dist_DewPoint_VC'])

    # - Si 'Dew Point (°C)' est plus proche des autres mesures que 'dewpoint' et 'VC_dew', alors 'Dew Point (°C)' est marqué comme plus proche.
    df_compare['DewPoint_closer'] = (df_compare['dist_dewpoint_DewPoint'] < df_compare['dist_dewpoint_VC']) & (df_compare['dist_dewpoint_DewPoint'] < df_compare['dist_DewPoint_VC'])

    # - Si 'VC_dew' est plus proche des autres mesures que 'dewpoint' et 'Dew Point (°C)', alors 'VC_dew' est marqué comme plus proche.
    df_compare['VC_closer'] = (df_compare['dist_DewPoint_VC'] < df_compare['dist_dewpoint_VC']) & (df_compare['dist_DewPoint_VC'] < df_compare['dist_dewpoint_DewPoint'])

    # Calculer les probabilités que chaque mesure soit la plus proche des autres sur l'ensemble des lignes :
    # La probabilité est simplement la proportion de fois où une mesure a été plus proche des autres.
    prob_dewpoint_closer = df_compare['dewpoint_closer'].mean().round(3)
    prob_dewpoint_c_closer = df_compare['DewPoint_closer'].mean().round(3)
    prob_vc_closer = df_compare['VC_closer'].mean().round(3)

    # Afficher les résultats
    # Ces résultats indiquent la probabilité que chaque mesure soit la plus proche des autres sur toutes les lignes de données
    print(f"Probability that 'dewpoint' is closer to the truth: {prob_dewpoint_closer}")
    print(f"Probability that 'Dew Point (°C)' is closer to the truth: {prob_dewpoint_c_closer}")
    print(f"Probability that 'VC_dew' is closer to the truth: {prob_vc_closer}")
    
except Exception as e:
    print(e)

In [ ]:
try:
    df_final['Dew Point (°C)'] = df_final['dewpoint']
    df_final = drop_columns_if_exist(df_final, ['dewpoint'])
    
except Exception as e:
    print(e)
display_row_values(df_final)

Rename Final Columns

In [ ]:
try:
    df_final.rename(columns={'Air T° Height': 'Air T° Height (m)',
                             'Barometer Elevation': 'Barometer Elevation (m)',
                             'Water T°': 'Water T° (°C)'}, inplace=True)

    print("Column 'Air T° Height' renamed to 'Air T° Height (m)'")

    print("Column 'Barometer Elevation' renamed to 'Barometer Elevation (m)'")
    print("Column 'Water T°' renamed to 'Water T° (°C)'")
    df_final['T°(C°)'] = df_final['T°(C°)'].round(2)

    print("Column 'T°(C°)' rounded to 2 decimal places")
except Exception as e:
    print(e)
    
display_row_values(df_final)

In [ ]:
df_final.dtypes

In [ ]:
# Correction des types des colonnes
try:
    df_final['Lat'] = df_final['Lat'].astype(str)
    df_final['Lon'] = df_final['Lon'].astype(str)
    df_final['Year'] = df_final['Year'].astype(str)
    print(df_final.dtypes)
except Exception as e:
    print(e)

In [ ]:
display_row_values(df_final)

In [ ]:
table_silver = "silver_table"
table_dim_station = "dim_station"
table_dim_time = "dim_time"
table_facts_meteo = "facts_meteo"
table_facts_ocean = "facts_ocean"

In [ ]:
display_row_values(df_final)

In [ ]:
df_final["Date ID"] = df_final["Datetime"].astype(str)

In [ ]:
try:
    # Supprimer la colonne 'Datetime'
    
    create_table_in_mysql(df=df_final, engine=engine, table_name=table_silver)
except Exception as e:
    print(f"Error when creating table:\n {str(e)}")
try:
    insert_new_rows(df=df_final, engine=engine, table_name=table_silver, ref= 'Date ID')
except Exception as e:
    print(f"Error when inserting new rows:\n {str(e)}")

In [ ]:
display_row_values(df_final)

Création des DataFrames pour les tables

In [ ]:
df_final.dtypes

In [ ]:
# Colonnes dérivées
df_final['Date ID'] = df_final['Datetime'].dt.strftime('%Y%m%d%H')
df_final['Unique ID'] = df_final['Datetime'].dt.strftime('%Y%m%d%H%M') + df_final['Station ID']


Export to CSV

In [ ]:
save_concat_csv(df_final, csv_folder="csv", base_filename="Cleaned_Data_Ocean_Meteo_ETL")

In [ ]:
#imprimer toutes les colonnes numeriquesq
num_cols = df_final.select_dtypes(include=[np.number]).columns

#imprimer toutes les colonnes non-numeriques
non_num_cols = df_final.select_dtypes(exclude=[np.number]).columns
non_num_cols

In [ ]:
display_row_values(df_final)

In [ ]:
##################################### DimStation ##################################################################################################

df_station = df_final[[
    'Station ID', 'Station Zone', 'Lat', 'Lon'
]].copy().drop_duplicates()

###################################### DimTime ##################################################################################################

df_time = df_final[['Datetime', 'Year', 'Month', 'DayOfWeek', 'DayPeriod']].copy().drop_duplicates()

########################################## Facts Meteo #########################################################################

df_facts_meteo = df_final[[
    'Unique ID', # PK
    'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
    'Sea Level Pressure (hPa)', 'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
    "Cloud Cover (%)", 'Visibility (km)', 'Wind Speed (10m)', 'Wind Direction (°)',
    'Wind Gusts (km/h)', 'Barometer Elevation (m)', 'Air T° Height (m)',
    
    'Station ID', # FK
    'Datetime' # FK
]].copy().drop_duplicates()

########################################## Facts Ocean #########################################################################

df_facts_ocean = df_final[[
    'Unique ID', # PK
    'Wave Height (m)', 'Average Wave Period (s)', 'Dominant Wave Direction (°)',
    'Water T° (°C)', 'Water Depth (m)', 'Sea Temperature Depth (m)',
    
    'Station ID', # FK
    'Datetime'  # FK
]].copy().drop_duplicates()


In [ ]:
display_row_values(df_time)

In [ ]:
display_row_values(df_station)

In [ ]:
display_row_values(df_facts_meteo)

In [ ]:
display_row_values(df_facts_ocean)

In [ ]:
# Vérifier les doublons
print(f"Doublons dans df_station : {df_station.duplicated().sum()}")
print(f"Doublons dans df_time : {df_time.duplicated().sum()}")
print(f"Doublons dans df_facts_meteo : {df_facts_meteo.duplicated().sum()}")
print(f"Doublons dans df_facts_ocean : {df_facts_ocean.duplicated().sum()}")

# Vérification des valeurs de base
print(f"\nDescription des données de df_station : \n{df_station.describe()}")
print(f"\nDescription des données de df_time : \n{df_time.describe()}")
print(f"\nDescription des données de df_facts_meteo : \n{df_facts_meteo.describe()}")
print(f"\nDescription des données de df_facts_ocean : \n{df_facts_ocean.describe()}")

Checking dim_station DataFrame

In [ ]:
# check unique values
print(f"{df_station.shape[0]}\n\n{df_station.nunique()}")
display_row_values(df_station)

Création de la table dim_station et insertion des données

In [ ]:
try:
    create_table_in_mysql(df=df_station, engine=engine, table_name=table_dim_station)
    insert_new_rows(df=df_station, engine=engine, table_name=table_dim_station, ref='Station ID')

    save_concat_csv(df_station, csv_folder="csv", base_filename="Dim_Station_Data_Ocean_Meteo_ETL")

except Exception as e:
    print(e)

Checking dim_time DataFrame

In [ ]:
print(f"{df_time.shape[0]}\n\n{df_time.nunique()}")
display_row_values(df_time)

Création de la table dim_time et insertion des données

In [ ]:
try:
    create_table_in_mysql(df=df_time, engine=engine, table_name=table_dim_time)
    insert_new_rows(df=df_time, engine=engine, table_name=table_dim_time, ref='Datetime')

    save_concat_csv(df_station, csv_folder="csv", base_filename="Dim_Time_Data_Ocean_Meteo_ETL")
except Exception as e:
    print(e)

Checking Meteo Facts DataFrame

In [ ]:
print(f"{df_facts_meteo.shape[0]}\n\n{df_facts_meteo.nunique()}")
display_row_values(df_facts_meteo)

Création de la table facts_meteo et insertion des données

In [ ]:
try:
    create_table_in_mysql(df=df_facts_meteo, engine=engine, table_name=table_facts_meteo)
    insert_new_rows(df=df_facts_meteo, engine=engine, table_name=table_facts_meteo, ref='Datetime')

    save_concat_csv(df_station, csv_folder="csv", base_filename="Facts_Meteo_Data_Ocean_Meteo_ETL")

except Exception as e:
    print(e)

Checking Ocean Facts DataFrame

In [ ]:
df_final.columns

In [ ]:
print(f"{df_facts_ocean.shape[0]}\n\n{df_facts_ocean.nunique()}")
display_row_values(df_facts_ocean)

Création de la table facts_ocean et insertion des données

In [ ]:
try:
    create_table_in_mysql(df=df_facts_ocean, engine=engine, table_name=table_facts_ocean)
    insert_new_rows(df=df_facts_ocean, engine=engine, table_name=table_facts_ocean, ref='Datetime')

    save_concat_csv(df_station, csv_folder="csv", base_filename="Facts_Ocean_Data_Ocean_Meteo_ETL")
except Exception as e:
    print(e)

In [ ]:
# try:
#     # Initialiser le metadata
#     metadata = MetaData()
#     metadata.reflect(bind=engine)

#     # Récupérer les tables existantes
#     dim_station = metadata.tables['dim_station']
#     dim_time = metadata.tables['dim_time']
#     facts_meteo = metadata.tables['facts_meteo']
#     facts_ocean = metadata.tables['facts_ocean']

#     # Ajouter les clés étrangères aux tables de faits
#     ForeignKeyConstraint(['Station ID'], [dim_station.c['Station ID']], name='fk_meteo_station').create(facts_meteo, bind=engine)
#     ForeignKeyConstraint(['Datetime'], [dim_time.c['Datetime']], name='fk_meteo_time').create(facts_meteo, bind=engine)

#     ForeignKeyConstraint(['Station ID'], [dim_station.c['Station ID']], name='fk_ocean_station').create(facts_ocean, bind=engine)
#     ForeignKeyConstraint(['Datetime'], [dim_time.c['Datetime']], name='fk_ocean_time').create(facts_ocean, bind=engine)
# except Exception as e:
#     print(str(e))